In [28]:
import pandas as pd
import numpy as np
import re
import os
import time

raw_path = 'C:\\Finance_team\\raw_data' # Raw Data 저장 경로\n"
file_list = os.listdir('C:\\Finance_team\\raw_data')

product_id = ''
product_code = ''
date = ''

def is_target_product_ID(line):
    return line[17:19] == 'A0' and line[425:428] == 'BM3' and line[471:474] == '001'

for i in file_list:

    with open('{0}\\{1}'.format(raw_path, i), 'r', encoding='ISO-8859-1') as f:

        df_i = pd.DataFrame(columns = ['time', 'cur_p', 'exec_q', 'inv_t', 'l_lvl1_q', 's_lvl1_q', 'inv_l_q', 'inv_s_q', 's_or_l'])
        for line in f:
            if is_target_product_ID(line):
                product_id = line[53:64]
                product_code = line[35:47]
                date = line[27:35]
                print(product_id, product_code, date)
                break
                
        for line in f:
            if line[17:19] == 'A3' and line[22:34] == product_code: 
                 
                ##### A3 에서 필요한 정보는?
                # 0) 시간: line[:17]
                # 1) 현재가: line[40:48]
                # 2) 체결량: line[48:54]
                # 3) 최종매도매수구분코드: line[152:153]
                
                a3_dic = {'time': line[:17],
                           'cur_p': line[40:48],
                           'exec_q': line[48:54],
                           'inv_t': None,
                           'l_lvl1_q': None,
                           's_lvl1_q': None,
                           'inv_l_q': None,
                           'inv_s_q': None,
                           's_or_l': line[152:153]}
                
                df_i.loc[len(df_i)] = a3_dic
                
                
            elif line[17:19] == 'G7' and line[22:34] == product_code:
                
                ##### G7 에서 필요한 정보는?
                # 0) 시간: line[:17]
                # 1) 현재가:line[40:48]
                # 2) 체결량: line[48:54]
                # 3) 매수1단계우선호가잔량 (l_lvl1_q) : line[169:175]
                # 4) 매도1단계우선호가잔량 (s_lvl1_q) : line[251:257]
                # 5) 최종매도매수구분코드 (s_or_l) : line[152:153]
                
                g7_dic = {'time': line[:17],
                           'cur_p': line[40:48],
                           'exec_q': line[48:54],
                           'inv_t': None,
                           'l_lvl1_q': line[169:175],
                           's_lvl1_q': line[251:257],
                           'inv_l_q': None,
                           'inv_s_q': None,
                           's_or_l': line[152:153]}
                
                df_i.loc[len(df_i)] = g7_dic                
            
            elif line[17:19] == 'B6' and line[22:34] == product_code:
                
                ##### B6 (우선호가) 에서 필요한 정보는?
                # 0) 시간: line[:17]
                # 1) 현재가: 없음 -> 따라서, 찾아올라가야 함.
                # 2) 매수1단계우선호가잔량: line[57:63]
                # 3) 매도1단계우선호가잔량: line[139:145]
                
                b6_dic = {'time': line[:17],
                           'cur_p': None,
                           'exec_q': None,
                           'inv_t': None,
                           'l_lvl1_q': line[57:63],
                           's_lvl1_q': line[139:145],
                           'inv_l_q': None,
                           'inv_s_q': None,
                           's_or_l': None,}   
                
                df_i.loc[len(df_i)] = b6_dic                 
                
            
            elif line[17:19] == 'H1' and (line[49:53] == '1000' or line[49:53] == '4000' or line[49:53] == '9000') and line[38:49] == product_id:
                
                ##### H1 에서 필요한 정보는?
                # 0) 시간: line[:17]
                # 1) 투자자유형: line[49:53]
                # 2) 매수약정수량: line[53:62]
                # 3) 매도약정수량: line[62:71]
                
                h1_dic = {'time': line[:17],
                           'cur_p': None,
                           'exec_q': None,
                           'inv_t': line[49:53],
                           'l_lvl1_q': None,
                           's_lvl1_q': None,
                           'inv_l_q': line[53:62],
                           'inv_s_q': line[62:71],
                           's_or_l': None,}                 
                
                df_i.loc[len(df_i)] = h1_dic                 
                         
                ##### 모든 코드 값을 정리해 보면, 내가 현재 필요한 Column은
                # 0) 시간 = time
                # 1) 현재가 = cur_p
                # 2) 체결량 = exec_q
                # 3) 투자자 유형 = inv_t
                # 4) 매수1단계우선호가 잔량 = l_lvl1_q
                # 5) 매도1호가 우선호가 잔량 = s_lvl1_q
                # 6) 매수약정수량 = inv_l_q
                # 7) 매도약정수량 = inv_s_q
                # 8) 최종매도매수구분코드 (1: 매도, 2: 매수) = s_or_l            
            
            
df_i.iloc[1300:1350, :]
                        
#         while not product_id:
#             line = f.readline()
#             if is_target_product_ID(line):
#                 product_id = line[53:64]
#                 product_code = line[35:47]
#                 date = line[27:35]
        
#         while True:
#             line = f.readline()
#             print(line)
            
#             break
            

# df = pd.read_csv('C:\\Finance_team\\raw_data\\sise2local.ALL_CMDT.0201', 'r', 
#                  encoding='iso-8859-1', header=None, names=None)

# df = df[0][(df[0].str.contains(product_code) & ((df[0].str.contains('A3')) | (df[0].str.contains('B6')) | (df[0].str.contains('G7')) | df[0].str.contains('H1')))]

# df = df.reset_index(drop=True)

# df_new = pd.DataFrame(columns = ['time', 'cur_price', 'short_lvl1_q', 'long_lvl1_q',
#                                   'short_exec_q', 'long_exec_q'])
# df[0][:17]

# df_new.loc[len(df_new)] = {'time': df[0][:17], ''}

KRDRVFUBM3  KR4165N30007 20180213


,time,cur_p,exec_q,inv_t,l_lvl1_q,s_lvl1_q,inv_l_q,inv_s_q,s_or_l
1300,[09:08:28-714146],None,None,None,000217,001318,None,None,None
1301,[09:08:28-716328],None,None,None,000220,001318,None,None,None
1302,[09:08:29-180683],None,None,None,000270,001318,None,None,None
1303,[09:08:29-527139],None,None,None,000300,001318,None,None,None
1304,[09:08:30-020991],None,None,None,000300,001318,None,None,None
1305,[09:08:30-028405],None,None,None,000300,001318,None,None,None
1306,[09:08:30-037170],None,None,None,000300,001318,None,None,None
1307,[09:08:30-044746],None,None,None,000300,001318,None,None,None
1308,[09:08:31-399161],None,None,None,000297,001318,None,None,None
1309,[09:08:31-401684],None,None,None,000294,001318,None,None,None
